# Определение рыночной стоимости автомобиля

В ходе данного проекта будет решаться задача об определении рычной стоимости автомобиля на синтетических данных вымышленной компании "Не бит, не крашен". В нашем распоряжении данные о технических характеристиках, комплектации и ценах автомобилей.<br>
Критерии, которые важны заказчику:
1. Качество предсказания
2. Время обучения модели
3. Время предсказания модели <br>

Для оценки качества моделей будем применять метрику RMSE, значение которой по условию заказчика должно быть меньше 2500.

## 1. Загрузка данных

Для начала подключим необходимые для выполнения работы библиотеки:

In [ ]:
import pandas as pd
import numpy as np
#!pip install missingno
import missingno as msno
#!pip install category_encoders
import category_encoders as ce
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
#!pip install lightgbm
from lightgbm import LGBMRegressor
#!pip install catboost
from catboost import CatBoostRegressor
from catboost import Pool, cv
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

Теперь загрузим данные:

In [ ]:
try:
    autos = pd.read_csv('autos.csv')
except:
    autos = pd.read_csv('/datasets/autos.csv')

## 2. Изучение данных

Осуществим первичный обзор датафрейма:

In [ ]:
autos.head()

Также получим основную информацию о данных:

In [ ]:
autos.info()

Изменим тип данных в столбце Repaired - вместо 'yes', 'no' будем использовать булевы значения True, False:

In [ ]:
autos = autos.replace({'Repaired': {'yes': True, 'no': False}})

Оптимизируем используемые типы числовых данных, чтобы улучшить использование памяти:

In [ ]:
for column in ['Price', 'RegistrationYear', 'Power', 'Kilometer', 'RegistrationMonth', 
               'NumberOfPictures', 'PostalCode', 'Repaired']:
    autos[column] = pd.to_numeric(autos[column], downcast='integer')

In [ ]:
autos.info(verbose=False)

Как видим, использование памяти сократилось больше, чем на четверть. 

Теперь проведем визуальный анализ пропущенных данных:

In [ ]:
msno.matrix(autos, sparkline=False)

Как видим, пропущенные значения встречаются в столбцах VehicleType, Gearbox, Model, FuelType и Repaired. Больше всего их в столбце Repaired, меньше всего - в Model. <br>
Заметим, что все пропуски у нас в категориальных данных. Предположим, что они связаны с ошибками в заполнении анкет или с ошибками выгрузки данных с сайта сервиса. <br>
Восстановить такого рода пропуски возможным не представляется, поэтому остается два возможных выхода - заменить их на какую-то "заглушку" (к примеру, вместо пропусков писать значение 'unknown') или удалить. Примем решение в пользу удаления:

In [ ]:
autos = autos.fillna('unknown')

Теперь, убрав пропущенные значения, можем присвоить столбцу Repaired булевский тип:

In [ ]:
autos['Repaired'] = autos['Repaired'].astype('bool')

Теперь проверим данные на наличие дубликатов:

In [ ]:
autos.duplicated().sum()

Нашли 4 дублированные строки, удалим их:

In [ ]:
autos = autos.drop_duplicates(ignore_index=True)

Проведем обзор основных статистик числовых признаков:

In [ ]:
autos.describe()

Сразу обратим внимание на то, что столбец NumberOfPictures полностью состоит из нулей (причем нулевым он является и в исходных данных, то есть, мы не удалили никаких существенных строк на предыдущих шагах). Удалим его:

In [ ]:
autos = autos.drop(['NumberOfPictures'], axis=1)

Кроме того, представляются несущественными для ценообразования такие признаки как DateCrawled, RegistrationYear, RegistrationMonth, DateCreated, PostalCode и LastSeen. <br>
Дата регистрации автомобиля не представляет большого значения в отличие от даты его производства. Дата создания и скачивания анкеты тем более не может играть роли в изменении его цены, то же верно и для даты последней активности пользователя, разместившего объявление, и для его почтового индекса. <br>
Обосновав их незначительность, удалим эти признаки:

In [ ]:
autos = autos.drop(['DateCrawled', 'RegistrationYear', 'RegistrationMonth', 'DateCreated', 
                    'PostalCode', 'LastSeen'], axis=1)

Теперь заметим необычно низкие значения в столбцах Price и Power. Взглянем на их наименьше значения более подробно, начав с признака Price:

In [ ]:
autos['Price'].value_counts().sort_index().head(10)

Видим неправдоподобно низкие даже для подержанных автомобилей цены. Отметим также весьма большое количество автомобилей, чья цена равна 0, и несколько меньшее, однако все еще выделяющееся число автомобилей с ценой в 1. Вероятно, эти значения говорят о желании продавца провести торг или обмен, но для определения рыночной цены такие значения не подходят. Оставим в нашем наборе лишь автомобили, цена которых не ниже 100 (число взято примерно после беглого анализа нескольких европейских онлайн-маркетплейсов подержанных авто): 

In [ ]:
autos = autos.query('Price >= 100')

Теперь взглянем на самые низкие значения признака Power:

In [ ]:
autos['Power'].value_counts().sort_index().head(10)

Видим ту же скученность данных в нуле. Удалим из набора все объекты, величина признака Power которых не превосходит 50 лошадиных сил:

In [ ]:
autos = autos.query('Power >= 50')

Так же посмотрим, с какими наибольшими значениями этого признака мы имеем дело:

In [ ]:
autos['Power'].value_counts(ascending=False).sort_index(ascending=False).head(10)

В этом случше тоже наблюдаем нереалистичные значения. Как известно автору, наибольшее число лошадиных сил среди автомобилей составляет примерно 2000. Также, среднее число лошадиных сил у спорткаров - в районе 500-1000. Таких машин в нашем датафрейме всего 91:   

In [ ]:
len(autos.query('Power >= 500 and Power <= 1000'))

Удалим все автомобили, значения Power которых превышает 1000:

In [ ]:
autos = autos.query('Power <= 1000')

На этом завершаем этап предобработки данных. 

## 3. Обучение моделей

Начнем обучение с разделения данных на выборки:

In [ ]:
autos.shape

In [ ]:
autos.info()

In [ ]:
autos['VehicleType'] = autos['VehicleType'].astype('category')
autos['Gearbox'] = autos['Gearbox'].astype('category')
autos['Model'] = autos['Model'].astype('category')
autos['FuelType'] = autos['FuelType'].astype('category')
autos['Brand'] = autos['Brand'].astype('category')

In [ ]:
autos.info()

In [ ]:
features = autos.drop('Price', axis=1)
target = autos['Price']

features_train, features_test, target_train, target_test = train_test_split(features, target, 
                                                          test_size=0.25, random_state=42)

Выведем размеры обучающей и тестовой выборок:

In [ ]:
print(features_train.shape, target_train.shape)
print(features_test.shape, target_test.shape)

Теперь определим RMSE с помощью использования MSE со значением False для параметра squared:

In [ ]:
rmse = make_scorer(mean_squared_error, squared=False, greater_is_better=False)

Начнем с модели библиотеки **LightGBM**:

Теперь создадим модель:

In [ ]:
model = LGBMRegressor(
    boosting_type='gbdt',
    num_leaves=31,
    max_depth=-1,
    learning_rate=0.1,
    n_estimators=100,
    objective='regression',
    min_split_gain=0.0,
    min_child_samples=20,
    subsample=1.0,
    subsample_freq=0,
    colsample_bytree=1.0,
    reg_alpha=0.0,
    reg_lambda=0.0,
    random_state=None
)

Проведем кросс-валидацию нашей модели:

In [ ]:
scores = cross_validate(model, features_train, target_train, cv=5, scoring=rmse)

Теперь получим значения метрики RMSE и количество времени, потраченного на обучение и предсказание:

In [ ]:
print('Значения RMSE на каждом шаге кросс-валидации и их среднее:')
print(abs(scores['test_score']))
print(abs(scores['test_score']).mean())
print('Время обучения на каждом шаге кросс-валидации и его среднее значение:')
print(abs(scores['fit_time']))
print(abs(scores['fit_time']).mean())
print('Время предсказания на каждом шаге кросс-валидации и его среднее значение:')
print(abs(scores['score_time']))
print(abs(scores['score_time']).mean())

Как видим, наша модель уже достаточно хорошо справляется со своей работой. Значение RMSE в каждом случае меньше 2500, средняя скорость обучения в данном эксперименте составила 5.7 секунд, а предсказания - 0.7 (значения могут незначительно меняться в каждом проведенном эксперименте)

Теперь проведем гридсерч по нескольким параметрам:

In [ ]:
parameters = {
              'max_depth': [15, 25, 35],
              'learning_rate': [0.01, 0.05, 0.1],
              'n_estimators': [50, 75, 100],
              'reg_alpha': [0, 1],
              'reg_lambda': [0, 1]
}

grid = GridSearchCV(estimator=model, param_grid=parameters, scoring=rmse, cv=5, verbose=3)

In [ ]:
grid.fit(features_train, target_train)

Выведем лучшие параметры для модели и ее усредненный лучший результат:

In [ ]:
print(grid.best_params_)
print(abs(grid.best_score_))

Как видим, лучший результат дают следующие параметры: learning_rate: 0.1, max_depth: 25, n_estimators: 100, reg_alpha: 0, reg_lambda: 0. Теперь посмотрим на средние характеристики времени для лучшей модели:

In [ ]:
pd.DataFrame(grid.cv_results_).loc[[grid.best_index_]]

Итак, лучший результат метрики RMSE для данной модели - 2251.4, время обучения - 4.3, время предказания - 0.6.

Теперь воспользуемся средствами библиотеки **CatBoost**:

In [ ]:
model = CatBoostRegressor()

In [ ]:
cat_features = ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand']

pool = Pool(data=features_train, 
            label=target_train, 
            cat_features=cat_features)

params = {'loss_function': 'RMSE',
          'iterations': 500,
          'depth': 3,
          }

scores = cv(pool=pool,
            params=params,
            fold_count=5,
            logging_level='Verbose',
           )

In [ ]:
scores['test-RMSE-mean'].min()

Как видим, величина RMSE слишком велика (лучшее среднее значение - 2607.9). Воспользуемся гридсерчем:

In [ ]:
param_grid = {'depth': [2, 6, 10],
              'l2_leaf_reg': [1, 5, 9]}

grid = model.grid_search(param_grid,
                         pool)

In [ ]:
grid['params']

Как видим, наилучшее значения для параметра depth - 10, а для l2_leaf_reg - 1.

In [ ]:
pd.DataFrame(grid['cv_results'])

Кроме того, видим, что к тысячной итерации среднее значение RMSE составляет 2150.8, что меньше порога в 2500. При использовании большего числа итераций, сможем достигнуть еще более низкого значения.

Теперь замерим время обучения и предсказания данных (не нашел способа сделать это лучше, так как из кросс-валидации и гридсерча CatBoost'а нельзя "достать" параметры времени, как из аналогичных инструментов sklearn'а):

In [ ]:
%%time
model.fit(X=features_train, y=target_train, cat_features=cat_features)

In [ ]:
%%time
model.predict(data=features_train)

Как видим, обучение модели занимает 9 минут 5 секунд, а предсказание - 2 секунды.

Также обучим **линейную регрессию**:

Сперва закодируем наши данные с помощью кодировки HashingEncoder:

In [ ]:
encoder = ce.HashingEncoder(cols=['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand'], n_components=32)
encoder.fit(features_train, target_train)
features_train_hash = encoder.transform(features_train)

Теперь создадим модель и проведем кросс-валидацию:

In [ ]:
model = LinearRegression()

In [ ]:
scores = cross_validate(model, features_train_hash, target_train, cv=5, scoring=rmse)

Теперь получим необходимые метрики:

In [ ]:
print('Значения RMSE на каждом шаге кросс-валидации и их среднее:')
print(abs(scores['test_score']))
print(abs(scores['test_score']).mean())
print('Время обучения на каждом шаге кросс-валидации и его среднее значение:')
print(abs(scores['fit_time']))
print(abs(scores['fit_time']).mean())
print('Время предсказания на каждом шаге кросс-валидации и его среднее значение:')
print(abs(scores['score_time']))
print(abs(scores['score_time']).mean())

Видим, что значение RMSE слишком велико - 3097. Время обучения составляет 0.91, а время предсказания - 0.12.

Наконец, проведем обучение модели **случайного леса**:

In [ ]:
model = RandomForestRegressor()

In [ ]:
parameters = {'random_state' : [12345],
              'n_estimators': range(60, 141, 40),
              'max_depth': [1, 5, 10]}

grid = GridSearchCV(estimator=model, param_grid=parameters, scoring=rmse, cv=3, verbose=3)

In [ ]:
grid.fit(features_train_hash, target_train)

Получим необходимые метрики:

In [ ]:
print(grid.best_params_)
print(abs(grid.best_score_))

In [ ]:
pd.DataFrame(grid.cv_results_).loc[[grid.best_index_]]

Наилучшее значение RMSE для случайного леса - 2544.5, среднее время обучения равняется 105.5,а время предсказания - 2.5.

Беря во внимание критерии, приоритетные для заказчика, делаем вывод, что лучшая для данной задачи модель - LGBMRegressor. Проведем ее тестирование, использовав подобранные параметры:

In [ ]:
model = LGBMRegressor(
    boosting_type='gbdt',
    num_leaves=31,
    max_depth=25,
    learning_rate=0.1,
    n_estimators=100,
    objective='regression',
    min_split_gain=0.0,
    min_child_samples=20,
    subsample=1.0,
    subsample_freq=0,
    colsample_bytree=1.0,
    reg_alpha=0.0,
    reg_lambda=0.0,
    random_state=None
)

In [ ]:
%%time
model.fit(features_train, target_train)

In [ ]:
%%time
target_predicted = model.predict(features_test)

In [ ]:
rmse = mean_squared_error(target_test, target_predicted, squared=False)
rmse

Итоговое тестирование показало значение RMSE в 2224.3, время обучения составило 2.5 секунд, а время предсказания - 390 миллисекунд. Результаты можем признать удовлетворительными.